In [ ]:
import polars as pl
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import category_encoders as ce

class TargetEncoderCV:
    def __init__(self, n_folds, seed, cv_type='kfold', groups=None):
        self.n_folds = n_folds
        self.seed = seed
        self.cv_type = cv_type
        self.groups = groups
        self.final_target_encoder = None

    def get_cv(self):
        if self.cv_type == 'kfold':
            return KFold(n_splits=self.n_folds, shuffle=True, random_state=self.seed)
        elif self.cv_type == 'stratified':
            return StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=self.seed)
        elif self.cv_type == 'group':
            if self.groups is None:
                raise ValueError("For GroupKFold, 'groups' must be provided.")
            return GroupKFold(n_splits=self.n_folds)
        else:
            raise ValueError("cv_type must be 'kfold', 'stratified', or 'group'.")

    def fit_transform(self, df, test, col, target_col):
        features = [c for c in df.columns if c != target_col]
        cv = self.get_cv()
        encoded_features = []

        for train_idx, val_idx in cv.split(df.to_pandas(), y=df[target_col].to_pandas() if self.cv_type == 'stratified' else None, groups=self.groups if self.cv_type == 'group' else None):
            X_train = df[train_idx, :]
            X_valid = df[val_idx, :]
            y_train = df[train_idx, target_col]

            target_encoder = ce.TargetEncoder()
            target_encoder.fit(X_train[col].to_pandas(), y_train.to_pandas())

            X_valid = X_valid.with_column(
                pl.Series(f'{col}_target_Encoded', target_encoder.transform(X_valid[col].to_pandas()))
            )
            encoded_features.append(X_valid)

        encoded_df = pl.concat(encoded_features).sort('index')
        df = df.with_column(encoded_df[f'{col}_target_Encoded'])

        # Fit on the entire dataset for the test data transformation
        self.final_target_encoder = ce.TargetEncoder()
        self.final_target_encoder.fit(df[col].to_pandas(), df[target_col].to_pandas())
        test = test.with_column(
            pl.Series(f'{col}_target_Encoded', self.final_target_encoder.transform(test[col].to_pandas()))
        )

        return df, test

    def transform(self, test, col):
        if self.final_target_encoder is None:
            raise ValueError("The model has not been fit yet. Call 'fit_transform' first.")
        test = test.with_column(
            pl.Series(f'{col}_target_Encoded', self.final_target_encoder.transform(test[col].to_pandas()))
        )
        return test

# groups = df['group_col'] 
# target_encoder_cv = TargetEncoderCV(n_folds=CFG.n_folds, seed=CFG.seed, cv_type='group', groups=groups)
# df_encoded, test_encoded = target_encoder_cv.fit_transform(df, test, col='col1', target_col='target')
# test_transformed = target_encoder_cv.transform(test, col='col1')
